In [1]:
!pip install tiledb
print("")

from datetime import datetime
import firecloud.api as FAPI
import json
import numpy as np
import os
import pandas as pd
import pprint
from tiledb import *


import platform
print("Python Version: ", platform.python_version())

pp = pprint.PrettyPrinter(indent=4)

BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
CATALOG_WORKSPACE_NAMESPACE = "anvil-datastorage"

# print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

print("")


# Print TileDB-Py version

print("TileDB Python API Version: ", tiledb.libtiledb.version())

# Print TileDB core version
print("TileDB Core Version:", tiledb.__version__)




print("I am ", FAPI.whoami())
print("Workspace Bucke: ", WORKSPACE_BUCKET)

## Read in tags list
!gsutil cp $WORKSPACE_BUCKET/AnVILCatalogWorkspacesPublicMetadataFromTerraAPI-2022-03-15.csv  .
workspacees = pd.read_csv('AnVILCatalogWorkspacesPublicMetadataFromTerraAPI-2022-03-15.csv',  sep="\t", keep_default_na=False)
# workspaceTags = workspaceTags[["name","sheet:consortium", "tagsSheet:tag:tags:dbGaP"]]
# workspacees.set_index("name", inplace=True)
len(workspacees)



array_url = WORKSPACE_BUCKET+"/tiledb/playground"
array_url = array_url.replace("gs","gcs")



Python Version:  3.7.12

TileDB Python API Version:  (2, 7, 1)
TileDB Core Version: 0.13.1
I am  pet-117272931645288568532@terra-e36fcccd.iam.gserviceaccount.com
Workspace Bucke:  gs://fc-cb5be780-171f-49d4-9116-b77fd2237d0b
Copying gs://fc-cb5be780-171f-49d4-9116-b77fd2237d0b/AnVILCatalogWorkspacesPublicMetadataFromTerraAPI-2022-03-15.csv...
/ [1 files][ 89.0 KiB/ 89.0 KiB]                                                
Operation completed over 1 objects/89.0 KiB.                                     


In [2]:
#https://gist.github.com/ihnorton/1544bb169b4ec8cd385e9c3e756c2bd5

# print(array_url)

# dataTypeDim  = tiledb.Dim(name="dataType", tile=None, dtype="ascii")
# indicationDim  = tiledb.Dim(name="indication", tile=None, dtype="ascii")
# dom = tiledb.Domain(indicationDim, dataTypeDim)
# attr = tiledb.Attr(name="workspace", dtype=np.str_)
# schema = tiledb.ArraySchema(domain=dom, sparse=True, attrs=[attr])

schema = ArraySchema(
  domain=Domain(*[
    Dim(name='workspace', domain=(None,None), tile=None, dtype=np.bytes_),
    Dim(name='consortium', domain=(None,None), tile=None, dtype=np.bytes_),
    Dim(name='phsId', domain=(None,None), tile=None, dtype=np.bytes_),
    Dim(name='library:indication', domain=(None,None), tile=None, dtype=np.bytes_), 
  ]),
  attrs=[
    Attr(name='name', dtype='<U0'),
  ],
  cell_order='row-major',
  tile_order='row-major', sparse=True)

## Well, blow me down!
tiledb.SparseArray.create(array_url, schema)





In [3]:
df = workspacees

def striplist(l):
    return([x.strip() for x in l])

# print(df)
with tiledb.open(array_url, 'w') as A:
    df_dict = {k: v.values for k,v in df.items()}   
    ws_dict = {
        'name': df_dict['name']
    }
    
    workspace = striplist(df_dict.pop('name'))
    consortium = striplist(df_dict.pop('consortium'))
    phsId = striplist(df_dict.pop('phsId'))
    ind = striplist(df_dict.pop('library:indication'))
    
    A[workspace, consortium,phsId, ind] = ws_dict

In [4]:
# val =A[:,:,:,'Mendelian disorders']

# qc = tiledb.QueryCondition("consortium == 'CMG'")

# set the qc object to the `attr_cond` argument in .query()
# q = A.query(attr_cond=qc)

# val = A.multi_index[:,:,:, ['Mendelian disorders', 'None']]
# data = A.multi_index[['1000G-high-coverage-2019', 'ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY'],:,:,'None']

# a_vals = data["name"]
# coords = data["coords"]

# print(a_vals)
# print(coords)



with tiledb.SparseArray(array_url, mode='r') as A:
    # Get non-empty domain
#     print("Non-empty domain: {}".format(A.nonempty_domain()))

    # Slice only rows 1, 2 and cols 2, 3, 4.
    # NOTE: The `query` syntax is required to specify an order
    # other than the default row-major
    data = A.query(attrs=["name"], coords=True)[:,'CCDG']
    print(data)
#     a_vals = data["name"]
#     coords = data["coords"]

#     for i in range(coords.shape[0]):
#         print("Cell {} has data {}".format(str(coords[i]), str(a_vals[i])))



OrderedDict([('name', array(['ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY',
       'AnVIL_CCDG_Baylor_CVD_AFib_BioVU_WGS',
       'AnVIL_CCDG_Baylor_CVD_AFib_Groningen_WGS',
       'AnVIL_CCDG_Baylor_CVD_AFib_VAFAR_HMB-IRB_WGS',
       'AnVIL_CCDG_Baylor_CVD_ARIC',
       'AnVIL_CCDG_Baylor_CVD_EOCAD_BioMe_WGS',
       'AnVIL_CCDG_Baylor_CVD_EOCAD_SoL_WGS',
       'AnVIL_CCDG_Baylor_CVD_HHRC_Brownsville_GRU_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_BNI_HMB_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_Duke_DS_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_ERICH_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_GERFHS_HMB_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_GOCHA_DS_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_Regards_DS_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_WashU_DS_WGS',
       'AnVIL_CCDG_Baylor_CVD_HemStroke_Yale_HMB_WGS',
       'AnVIL_CCDG_Baylor_CVD_Oregon_SUDS_GRU_WGS',
       'AnVIL_CCDG_Baylor_CVD_TexGen_DS_WGS',
       'AnVIL_CCDG_Baylor_CVD_Ventura_Presto_GRU-I

In [18]:
# A = tiledb.open(array_url)
val = A.multi_index[:,:,:, :]
qdf = pd.DataFrame(val)

print(qdf['library:indication'].value_counts().to_dict())
print(qdf['consortium'].value_counts().to_dict())


{b'epilepsy': 152, b'Mendelian disorders': 73, b'inflammatory bowel disease': 66, b'autism spectrum disorder': 51, b'atrial fibrillation': 45, b'Unspecified': 17, b'coronary artery disease': 16, b'cardiovascular system disease': 11, b'myocardial infarction': 9, b'hemorrhagic stroke': 8, b'pediatric disease': 5, b'None': 4, b'population': 4, b"Alzheimer's disease": 3, b'--': 3, b'Schizophrenia cases and controls': 3, b'heart and blood vessel disease': 2, b'Schizophrenia and Bipolar Disorder cases and controls': 2, b'atherosclerosis': 2, b'CIRM iPSC Repository': 1, b'spinal muscular atrophy': 1, b'type 1 diabetes mellitus': 1, b'epigenetic variation': 1, b'cardiovascular disease': 1, b'cardiovascular disease and risk factors': 1, b'asthma': 1, b'control': 1, b'NA': 1}
{b'CCDG': 375, b'CMG': 72, b'CSER': 7, b'GTEx': 6, b'CMH': 5, b'WGSPD1': 5, b'PAGE': 5, b'Convergent Neuroscience': 4, b'eMERGE': 3, b'1000G': 1, b'HPRC': 1, b'T2T': 1}


In [94]:
import tiledb.sql, pandas

db = tiledb.sql.connect()
pandas.read_sql(sql="select * from `array_url`", con=db)

ModuleNotFoundError: No module named 'tiledb.sql'

In [17]:
import tiledb, numpy as np, tempfile
# Write to multi-attribute 2D array
with tempfile.TemporaryDirectory() as tmp:
    dom = tiledb.Domain(
        tiledb.Dim(name="y", domain=(None,None), tile=None, dtype=np.bytes_),
        tiledb.Dim(name="x", domain=(None,None), tile=None, dtype=np.bytes_))
    schema = tiledb.ArraySchema(domain=dom, sparse=True,
         attrs=(tiledb.Attr(name="a1", dtype=np.int64),
            tiledb.Attr(name="a2", dtype=np.int64)))
    tiledb.SparseArray.create(tmp + "/array", schema)
    with tiledb.SparseArray(tmp + "/array", mode='w') as A:
        # Write in the twp cells (0,0) and (2,3) only.
        I, J = ['0', '2'], ['0', '3']
        # Write to each attribute
        A[I, J] = {"a1": np.array([1, 2]),
                "a2": np.array([3, 4])}
    with tiledb.SparseArray(tmp + "/array", mode='r') as A:
        # Return an OrderedDict with values and coordinates
        data = A.multi_index['0', '0']
        print(data)
        # Return just the "x" coordinates values
        data=A.multi_index['0':'3', '0':'10']["x"]
        print(data)
            

OrderedDict([('y', array([b'0'], dtype=object)), ('x', array([b'0'], dtype=object)), ('a1', array([1])), ('a2', array([3]))])
[b'0']


In [37]:
df = pd.read_csv('AnVILCatalogWorkspacesPublicMetadataFromTerraAPI-2022-03-15.csv',  sep="\t")
schema = ArraySchema(
  domain=Domain(*[
    Dim(name='workspace', domain=(None,None), tile=None, dtype=np.bytes_),
    Dim(name='consortium', domain=(None,None), tile=None, dtype=np.bytes_),
    Dim(name='phsId', domain=(None,None), tile=None, dtype=np.bytes_),
    Dim(name='library:indication', domain=(None,None), tile=None, dtype=np.bytes_), 
  ]),
  attrs=[
    Attr(name='name', dtype='<U0'),
  ],
  cell_order='row-major',
  tile_order='row-major', sparse=True)
with tempfile.TemporaryDirectory() as tmp:
    tiledb.SparseArray.create(tmp + "/array", schema)
    with tiledb.SparseArray(tmp + "/array", mode='w') as A:
        df_dict = {k: v.values for k,v in df.items()}   
        ws_dict = {
            'name': df_dict['name']
        }

        workspace = df_dict.pop('name')
        consortium = df_dict.pop('consortium')
        phsId = df_dict.pop('phsId')
        ind = df_dict.pop('library:indication')

        A[workspace, consortium,phsId, ind] = ws_dict
        
    with tiledb.SparseArray(tmp + "/array", mode='r') as A:
        val = A.multi_index[:,[' T2T', ' 1000G'],:,['None']]
        print(pd.DataFrame(val))
    

                     workspace consortium     phsId library:indication  \
0                 b'AnVIL_T2T'    b' T2T'  b' None'            b'None'   
1  b'1000G-high-coverage-2019'  b' 1000G'  b' None'            b'None'   

                       name  
0                 AnVIL_T2T  
1  1000G-high-coverage-2019  


In [58]:
tiledb.remove(array_url)